In [2]:
from lxml import html
import requests

Get list of all datasets from PO.DAAC at JPL using the max items of 400 per request and running until we get a response containing less than 400 entries

In [3]:
iteration = 0
items_per_iteration = 400
entry_list = []
while True:
    response = requests.get("https://podaac.jpl.nasa.gov/ws/search/dataset/?startIndex=" + str(iteration * items_per_iteration) + "&itemsPerPage=" + str(items_per_iteration) + "&format=html")
    html_tree = html.fromstring(response.content)
    entries = html_tree.cssselect("div.entry")
    entry_list += entries
    if len(entries) < items_per_iteration:
        break
    iteration += 1

Creates a list of datasets and their storage location

In [4]:
ftps = []
unknowns = []
for entry in entry_list:
    found = False
    links = entry.cssselect("div.links")
    for link in links:
        for child in link.getchildren():
            if "FTP URL" in child.text:
                found = True
                ftps.append({"dataset" : entry.getchildren()[0].text, "url" : child.attrib['href']})
    if not found:
        unknowns.append(entry.getchildren()[0].text)

In [5]:
unknowns

['MODIS Aqua Level 3 Global Monthly Mapped 4 km Chlorophyll a',
 'MODIS Aqua Level 3 Global Daily Mapped 4 km Chlorophyll a',
 'SeaWiFS Global Daily Mapped 9km Chlorophyll a',
 'SeaWiFS Global Monthly Mapped 9 km Chlorophyll a',
 'MODIS Aqua Level 3 Global Daily Mapped 4 km Chlorophyll a v2014.0',
 'MODIS Aqua Level 3 Global Monthly Mapped 4 km Chlorophyll a v2014.0']

In [6]:
ftps

[{'dataset': 'OSTM GPS based orbit and SSHA OGDR',
  'url': 'ftp://podaac-ftp.jpl.nasa.gov/allData/ostm/preview/L2/GPS-OGDR'},
 {'dataset': 'GHRSST Level 4 RTO Aqua MODIS-AMSRE Day North America Regional Blended Sea Surface Temperature Analysis',
  'url': 'ftp://podaac-ftp.jpl.nasa.gov/allData/ghrsst/data/L4/NCAMERICA/JPL/RTO_SST_Ad'},
 {'dataset': 'SeaWinds on ADEOS-II Level 3 Sigma-0 Polar-Stereographic Browse Images of Antarctica',
  'url': 'ftp://podaac-ftp.jpl.nasa.gov/allData/seawinds/L3/byu_scp/sigma0browse/images/antarctic'},
 {'dataset': 'SeaWinds on ADEOS-II Level 3 Sigma-0 Polar-Stereographic Browse Maps (Reduced) of Antarctica',
  'url': 'ftp://podaac-ftp.jpl.nasa.gov/allData/seawinds/L3/byu_scp/sigma0browse/lite/antarctic'},
 {'dataset': 'Cross-Calibrated Multi-Platform Ocean Surface Wind Vector L3.5A Pentad First-Look Analyses',
  'url': 'ftp://podaac-ftp.jpl.nasa.gov/allData/ccmp/L3.5a/pentad/flk'},
 {'dataset': 'SeaWinds on QuikSCAT Level 3 Sigma-0 Polar-Stereographic B

Go to each url and crawl through the data for files (Still working on this)

In [7]:
from ftplib import FTP
from urllib.parse import urlparse

In [8]:
parsed = urlparse(ftps[0]['url'])

In [9]:
ftp = FTP(parsed.netloc)
ftp.login()

'230 Login successful.'

In [10]:
def traverse_ftp_directory(ftp, directory):
    current_directory = ftp.pwd()
    files = []
    try:
        ftp.cwd(directory)
        items = ftp.nlst()
        for item in items:
            files += traverse_ftp_directory(current_directory + "/" + item)
        return files
    except Exception:
        return [directory]

In [11]:
traverse_ftp_directory(ftp, "/allData/ghrsst/data/L2P/TMI/REMSS")

['/allData/ghrsst/data/L2P/TMI/REMSS']

Testing some ideas

In [12]:
files = []
try:
    ftp.cwd("/allData/ghrsst/data/L2P/TMI/REMSS/2009/")
    items = ftp.nlst()
    for item in items:
        files.append("/allData/ghrsst/data/L2P/TMI/REMSS" + "/" + item)
except Exception:
    print("Done")

In [13]:
files

['/allData/ghrsst/data/L2P/TMI/REMSS/155',
 '/allData/ghrsst/data/L2P/TMI/REMSS/203',
 '/allData/ghrsst/data/L2P/TMI/REMSS/215',
 '/allData/ghrsst/data/L2P/TMI/REMSS/235',
 '/allData/ghrsst/data/L2P/TMI/REMSS/236',
 '/allData/ghrsst/data/L2P/TMI/REMSS/238',
 '/allData/ghrsst/data/L2P/TMI/REMSS/295',
 '/allData/ghrsst/data/L2P/TMI/REMSS/301',
 '/allData/ghrsst/data/L2P/TMI/REMSS/331']